<h3><strong>Porównanie algorytmu: Minimax vs Minimax z Alpha-Beta pruning</strong></h3>

In [61]:
import chess
import chess.svg
from IPython.display import display, SVG
import time
from chess.polyglot import zobrist_hash
from collections import namedtuple

# Tabela wartości figur
piece_values = {
    chess.PAWN: 100,
    chess.KNIGHT: 300,
    chess.BISHOP: 300,
    chess.ROOK: 500,
    chess.QUEEN: 900,
    chess.KING: 0
}

piece_position = {
    chess.PAWN: [
        0,  0,  0,  0,  0,  0,  0,  0,
        50, 50, 50, 50, 50, 50, 50, 50,
        10, 10, 20, 30, 30, 20, 10, 10,
        5,  5, 10, 10, 10, 10,  5,  5,
        0,  0,  0, 20, 20,  0,  0,  0,
        5, -5,-10,  0,  0,-10, -5,  5,
        5, 10, 10,-20,-20, 10, 10,  5,
        0,  0,  0,  0,  0,  0,  0,  0
    ],

    chess.KNIGHT: [
        -50,-40,-30,-30,-30,-30,-40,-50,
        -40,-20,  0,  0,  0,  0,-20,-40,
        -30,  0, 10, 15, 15, 10,  0,-30,
        -30,  5, 15, 20, 20, 15,  5,-30,
        -30,  0, 15, 20, 20, 15,  0,-30,
        -30,  5, 10, 15, 15, 10,  5,-30,
        -40,-20,  0,  5,  5,  0,-20,-40,
        -50,-40,-30,-30,-30,-30,-40,-50,
    ],

    chess.BISHOP: [
        -20,-10,-10,-10,-10,-10,-10,-20,
        -10,  0,  0,  0,  0,  0,  0,-10,
        -10,  0,  5, 10, 10,  5,  0,-10,
        -10,  5,  5, 10, 10,  5,  5,-10,
        -10,  0, 10, 10, 10, 10,  0,-10,
        -10, 10, 10, 10, 10, 10, 10,-10,
        -10,  5,  0,  0,  0,  0,  5,-10,
        -20,-10,-10,-10,-10,-10,-10,-20,
    ],

    chess.ROOK: [
        0,  0,  0,  0,  0,  0,  0,  0,
        5, 10, 10, 10, 10, 10, 10,  5,
        -5,  0,  0,  0,  0,  0,  0, -5,
        -5,  0,  0,  0,  0,  0,  0, -5,
        -5,  0,  0,  0,  0,  0,  0, -5,
        -5,  0,  0,  0,  0,  0,  0, -5,
        -5,  0,  0,  0,  0,  0,  0, -5,
        0, -50,  0,  5,  5,  0, -50, 0
    ],

    chess.QUEEN: [
        -20,-10,-10, -5, -5,-10,-10,-20,
        -10,  0,  0,  0,  0,  0,  0,-10,
        -10,  0,  5,  5,  5,  5,  0,-10,
        -5,  0,  5,  5,  5,  5,  0, -5,
        0,  0,  5,  5,  5,  5,  0, -5,
        -10,  5,  5,  5,  5,  5,  0,-10,
        -10,  0,  5,  0,  0,  0,  0,-10,
        -20,-10,-10, -5, -5,-10,-10,-20
    ],
    
    chess.KING: [
        -30,-40,-40,-50,-50,-40,-40,-30,
        -30,-40,-40,-50,-50,-40,-40,-30,
        -30,-40,-40,-50,-50,-40,-40,-30,
        -30,-40,-40,-50,-50,-40,-40,-30,
        -20,-30,-30,-40,-40,-30,-30,-20,
        -10,-20,-20,-20,-20,-20,-20,-10,
        20, 20,  0,  0,  0,  0, 20, 20,
        20, 30, 10,  0,  0, 10, 30, 20
    ]
}


In [62]:
# def evaluate_board(position):
#     score = 0
#     for square in chess.SQUARES:
#         piece = position.piece_at(square)
#         if piece:
#             piece_type = piece.piece_type
#             if piece.color == chess.WHITE:
#                 mirrored = chess.square_mirror(square)
#                 score += piece_values[piece_type]
#                 score += piece_position[piece_type][mirrored] 
#             else:
#                 score -= piece_values[piece_type]
#                 score -= piece_position[piece_type][square]
#     return score/100

In [63]:
def evaluate_board(board):
    piece_values = {
        chess.PAWN: 1,
        chess.KNIGHT: 3,
        chess.BISHOP: 3,
        chess.ROOK: 5,
        chess.QUEEN: 9,
        chess.KING: 0
    }
    score = 0
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            value = piece_values[piece.piece_type]
            if piece.color == chess.WHITE:
                score += value
            else:
                score -= value
    return score

In [ ]:
import chess
import time
from collections import namedtuple, OrderedDict
from chess.polyglot import zobrist_hash, POLYGLOT_RANDOM_ARRAY

# Klasa z inkrementalnym hashem
class ZobristBoard(chess.Board):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.zobrist_hash = zobrist_hash(self)
        self.hash_stack = [self.zobrist_hash]

    def _piece_hash(self, piece, square):
        color_offset = 0 if piece.color == chess.WHITE else 6
        type_offset = piece.piece_type - 1
        return POLYGLOT_RANDOM_ARRAY[64 * (color_offset + type_offset) + square]

    def push(self, move):
        h = self.zobrist_hash

        from_square = move.from_square
        to_square = move.to_square
        moved_piece = self.piece_at(from_square)

        # Usuń przesuniętą figurę z pola
        h ^= self._piece_hash(moved_piece, from_square)

        # Zajmij się zdobytą figurą
        captured = self.piece_at(to_square)
        if captured:
            h ^= self._piece_hash(captured, to_square)

        # Dodaj przesunięty element do pola (lub awansuj)
        if move.promotion:
            promoted_piece = chess.Piece(move.promotion, moved_piece.color)
            h ^= self._piece_hash(promoted_piece, to_square)
        else:
            h ^= self._piece_hash(moved_piece, to_square)

        # En passant
        if self.is_en_passant(move):
            ep_square = to_square - 8 if self.turn == chess.WHITE else to_square + 8
            ep_pawn = chess.Piece(chess.PAWN, not self.turn)
            h ^= self._piece_hash(ep_pawn, ep_square)

        # Roszady
        if self.is_castling(move):
            if self.turn == chess.WHITE:
                if to_square == chess.G1:
                    rook_from, rook_to = chess.H1, chess.F1
                else:
                    rook_from, rook_to = chess.A1, chess.D1
            else:
                if to_square == chess.G8:
                    rook_from, rook_to = chess.H8, chess.F8
                else:
                    rook_from, rook_to = chess.A8, chess.D8
            rook = chess.Piece(chess.ROOK, self.turn)
            h ^= self._piece_hash(rook, rook_from)
            h ^= self._piece_hash(rook, rook_to)

        # Zmiana stron
        h ^= POLYGLOT_RANDOM_ARRAY[780]

        # Obsługa starych roszad i en passant
        old_castling_xors = 0
        if self.has_kingside_castling_rights(chess.WHITE):
            old_castling_xors ^= POLYGLOT_RANDOM_ARRAY[768]
        if self.has_queenside_castling_rights(chess.WHITE):
            old_castling_xors ^= POLYGLOT_RANDOM_ARRAY[769]
        if self.has_kingside_castling_rights(chess.BLACK):
            old_castling_xors ^= POLYGLOT_RANDOM_ARRAY[770]
        if self.has_queenside_castling_rights(chess.BLACK):
            old_castling_xors ^= POLYGLOT_RANDOM_ARRAY[771]

        old_ep_xor = 0
        if self.ep_square:
            old_ep_xor ^= POLYGLOT_RANDOM_ARRAY[772 + chess.square_file(self.ep_square)]

        # Zrób ruch
        super().push(move)

        # Obsługa nowych roszad i en passant
        new_castling_xors = 0
        if self.has_kingside_castling_rights(chess.WHITE):
            new_castling_xors ^= POLYGLOT_RANDOM_ARRAY[768]
        if self.has_queenside_castling_rights(chess.WHITE):
            new_castling_xors ^= POLYGLOT_RANDOM_ARRAY[769]
        if self.has_kingside_castling_rights(chess.BLACK):
            new_castling_xors ^= POLYGLOT_RANDOM_ARRAY[770]
        if self.has_queenside_castling_rights(chess.BLACK):
            new_castling_xors ^= POLYGLOT_RANDOM_ARRAY[771]

        new_ep_xor = 0
        if self.ep_square:
            new_ep_xor ^= POLYGLOT_RANDOM_ARRAY[772 + chess.square_file(self.ep_square)]

        h ^= old_castling_xors ^ new_castling_xors
        h ^= old_ep_xor ^ new_ep_xor

        self.zobrist_hash = h
        self.hash_stack.append(h)

    def pop(self):
        super().pop()
        self.hash_stack.pop()
        self.zobrist_hash = self.hash_stack[-1]

# LRU Cache
class LRUCache(OrderedDict):
    def __init__(self, maxsize):
        self.maxsize = maxsize
        super().__init__()

    def __getitem__(self, key):
        value = super().__getitem__(key)
        self.move_to_end(key)
        return value

    def __setitem__(self, key, value):
        super().__setitem__(key, value)
        self.move_to_end(key)
        if len(self) > self.maxsize:
            self.popitem(last=False)

Entry = namedtuple('Entry', ['value', 'depth', 'flag', 'best_move'])

transposition_table = LRUCache(maxsize=100000)

eval_count_with_tt = 0
def minimax_with_tt_opt(position, depth, alpha, beta, maximizingPlayer):
    global eval_count_with_tt
    state_key = position.zobrist_hash
    
    if state_key in transposition_table:
        entry = transposition_table[state_key]
        if entry.depth >= depth:
            if entry.flag == 'exact':
                return entry.value, entry.best_move
            elif entry.flag == 'lowerbound' and entry.value >= beta:
                return entry.value, entry.best_move
            elif entry.flag == 'upperbound' and entry.value <= alpha:
                return entry.value, entry.best_move
    
    if depth == 0 or position.is_game_over():
        eval_count_with_tt += 1
        value = evaluate_board(position)
        transposition_table[state_key] = Entry(value, depth, 'exact', None)
        return value, None
    
    alpha_orig = alpha
    beta_orig = beta
    best_move = None
    
    # Sortowanie ruchów
    moves = list(position.legal_moves)
    if state_key in transposition_table and transposition_table[state_key].best_move is not None and transposition_table[state_key].best_move in moves:
        best_from_tt = transposition_table[state_key].best_move
        moves.remove(best_from_tt)
        moves.insert(0, best_from_tt)
    
    if maximizingPlayer:
        maxEval = float('-inf')
        for move in moves:
            position.push(move)
            evaluation, _ = minimax_with_tt_opt(position, depth-1, alpha, beta, False)
            position.pop()
            if evaluation > maxEval:
                maxEval = evaluation
                best_move = move
            alpha = max(alpha, evaluation)
            if beta <= alpha:
                break
        value = maxEval
    else:
        minEval = float('inf')
        for move in moves:
            position.push(move)
            evaluation, _ = minimax_with_tt_opt(position, depth-1, alpha, beta, True)
            position.pop()
            if evaluation < minEval:
                minEval = evaluation
                best_move = move
            beta = min(beta, evaluation)
            if beta <= alpha:
                break
        value = minEval
    
    flag = 'exact'
    if value <= alpha_orig:
        flag = 'upperbound'
    elif value >= beta_orig:
        flag = 'lowerbound'
    
    transposition_table[state_key] = Entry(value, depth, flag, best_move)
    
    return value, best_move

# Test
board = ZobristBoard()

# Z TT, depth=6, 10 moves
board.reset()
white = True
move_count = 0
start = time.time()
while not board.is_game_over() and move_count < 10:
    score, move = minimax_with_tt_opt(board, 6, float('-inf'), float('inf'), white)
    if move is None:
        break
    board.push(move)
    white = not white
    move_count += 1
end = time.time()
print(f"Czas dla gry z TT opt: {end - start:.2f} s")

Czas dla gry z TT opt: 80.51 s
